# An Example of Real-Time DAS Processing Using a Rolling Mean Function

This Jupyter Notebook is created to showcase rolling mean processing on a [spool](https://dascore.org/tutorial/concepts.html#:~:text=read%20the%20docs!-,Data%20structures,-DASCore%20has%20two) of distributed acoustic sensing (DAS) data in real-time.
 

In [ ]:
# Import libraries
import os
import time
import warnings

import dascore as dc
import numpy as np

from dascore.units import s  # s as seconds

warnings.simplefilter('ignore')


### Get a spool of data to work on

In [ ]:
# Define data path (spool of data) and output folder 
data_path = '/mnt/h/data'
output_data_folder =  '/mnt/h/results'

# Get the sorted spool of data form the defined data path (on first run, it will index the patches and subsequently update the index file for future uses)
sp = dc.spool(data_path).sort("time").update()

# Print the contents of first 5 patches
content_df = sp.get_contents()
content_df.head()

### Get some metadata and set real-time processing parameters

In [ ]:
# Get sampling rate, channel spacing, and gauge length from the first patch
patch_0 = sp[0]

gauge_length = patch_0.attrs['gauge_length']
print(f"Gauge length = {gauge_length}")

channel_spacing = patch_0.attrs['d_distance']
print(f"Channel spacing = {channel_spacing}")

sampling_interval = patch_0.attrs['d_time']
print(f"Sampling interval = {sampling_interval}")

sampling_rate = 1/(sampling_interval / np.timedelta64(1, 's'))
print(f"Sampling rate = {sampling_rate}")

num_sec = len(patch_0.coords["time"]) / sampling_rate
print(f"Number of seconds in each patch = {num_sec}")

# Define the target sampling interval (in sec.)
d_t = 10 # Here, cutoff_freq = Nyq_new = 1/(2*d_t) = 0.05 hz

# Determine window size (in sec.)
window = d_t*s

# Determine step size (in sec.)
step = d_t*s

# Set the desired wait time after each run (in sec.)
time_step_for_processing = 60 


### Rolling mean processing in real-time

In [ ]:
# Start the for loop for real-time processing
i = 0
while True:
    initial_run = (i == 0)
    run_num = i+1
    print(f"\nRun number: {run_num}")

    # Select a updated sub-spool
    sp = dc.spool(data_path).sort("time").update()
    len_updated_sp = len(sp)

    # Break the while loop if there are no new patches in the spool
    if not initial_run and len_last_sp == len_updated_sp: # sleep first and then break
        time.sleep(4*time_step_for_processing)    

        sp = dc.spool(data_path).sort("time").update()
        len_updated_sp = len(sp)

        print("No new data was detected in spool after . "
            "Real-time data processing ended successfully.")
        break

    # Do processing on each patch in the spool
    for j, patch in enumerate (sp[len_last_sp:]): # check
        print(f"Working on patch number: {j}")

        rolling_mean_patch = patch.rolling(time=window, step=step, engine="numpy").mean()

        # Save results after processing
        file_name = sp.get_contents()["path"][j]
        output_path = os.path.join(output_data_folder, file_name)
        rolling_mean_patch.io.write(output_path, "dasdae")

    len_last_sp = len(sp)
    i+=1

    # Wait for new data to get into the data_path before proceeding with a new run
    time.sleep(time_step_for_processing)
